# Spark Cluster Master Node termination

### Importing Required Libraries ::
* __boto3__: Required to connect to AWS
* __botocore__: Required to handle the exceptions related to boto3 tasks

In [1]:
import boto3
from botocore.exceptions import ClientError

### Declaring the required hardcoded informations ::

In [2]:
user = 'root'
instance_termination_flag = 0

### Creating boto3 session, clients and resources ::

In [3]:
try:
    session = boto3.session.Session(region_name='us-east-1')
    ec2_client = session.client('ec2')
    ec2_resource = session.resource('ec2')
except ClientError as e:
    print("Unexpected error while creating boto3 session, client and resources: " + str(e))
    exit()

### Extract details of running Slave Nodes for current user ::
* If there is any Slave Node running for current user, Master Node will not be terminated.
* In that case please terminate all Slave Nodes using 'slave_nodes_termination.ipynb' first the re-run this Notebook.

In [4]:
try:
    instances = ec2_resource.instances.filter(
        Filters=[
            {
                'Name': 'instance-state-name',
                'Values': ['running']
            },
            {
                'Name': 'tag:Project',
                'Values': ['SparkCluster']
            },
            {
                'Name': 'tag:User',
                'Values': [user]
            },
            {
                'Name': 'tag:NodeType',
                'Values': ['Slave']
            }
        ]
    )
    if list(instances):
        print("Some Slave Nodes are in running state for User('" + user + "'). Master Node can not be terminated while any Slave Node is in 'running' state. Running slave nodes are:")
        print(list(instances))
        exit()
    else:
        print("No slave node is available is running state for User('" + user + "'). Starting to terminate master node.")
        instance_termination_flag = 1
except ClientError as e:
    print("Unexpected error while looking for already running Master node EC2 instance for user-'" + user + "': " + str(e))
    exit()

No slave node is available is running state for User('root'). Starting to terminate master node.


### Extract details of running Master Node for current user ::
* Extracted master node will be terminated. Any running process in the same node will be terminated automatically.

In [5]:
if instance_termination_flag:
    try:
        instances = ec2_resource.instances.filter(
            Filters=[
                {
                    'Name': 'instance-state-name',
                    'Values': ['running']
                },
                {
                    'Name': 'tag:Project',
                    'Values': ['SparkCluster']
                },
                {
                    'Name': 'tag:User',
                    'Values': [user]
                },
                {
                    'Name': 'tag:NodeType',
                    'Values': ['Master']
                }
            ]
        )
    except ClientError as e:
        print("Unexpected error while looking for already running Master node EC2 instance for user-'" + user + "': " + str(e))
        exit()

### Terminating the EC2 for master node on AWS ::
* __terminate__ API is used under EC2 resource to terminate the EC2 node, which is Master Node of our spark cluster.

In [6]:
if instance_termination_flag:
    if list(instances):
        try:
            for instance in instances:
                instance.terminate()
                print("Master node(InstanceId-'" + str(instance.id) + "') for user-'" + user + "' is pushed to termination. It will be terminated within 2-3 mins.")
        except ClientError as e:
            print("Unexpected error while creating Spark Cluster Master node EC2 instance for user-'" + user + "': " + str(e))
            exit()
    else:
        print("There is no running master node for user-'" + user + "'.")

Master node(InstanceId-'i-0a27306c67986fd4b') for user-'root' is pushed to termination. It will be terminated within 2-3 mins.
